# Carbon Capture

## Chemical Reaction ( Part 1 Carbamic Acid )

$$NH_3 + CO_2 \rightarrow NH_{2}COOH$$

first of all we want to model the vibrational ground state enegies of the reactants using VQE

In [8]:
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.drivers import PySCFDriver

driver = PySCFDriver(
    atom="C 0 0 0; O 0 0 1.162000; O 0 0 -1.162000",
    basis="sto3g",
    charge=0,
    spin=0,
    unit=DistanceUnit.ANGSTROM,
)

es_problem = driver.run()

In [9]:
from qiskit_nature.second_q.transformers import ActiveSpaceTransformer

transformer = ActiveSpaceTransformer(
    num_electrons=4,
    num_spatial_orbitals=4
)
es_problem = transformer.transform(es_problem)

In [10]:
from qiskit_nature.second_q.mappers import JordanWignerMapper

mapper = JordanWignerMapper()

In [11]:
from qiskit_algorithms import NumPyMinimumEigensolver

numpy_solver = NumPyMinimumEigensolver()

In [12]:
from qiskit_algorithms import VQE
from qiskit.primitives import StatevectorEstimator
from qiskit_algorithms.optimizers import SLSQP
from qiskit_nature.second_q.circuit.library import HartreeFock, UCCSD


ansatz = UCCSD(
    es_problem.num_spatial_orbitals,
    es_problem.num_particles,
    mapper,
    initial_state=HartreeFock(
        es_problem.num_spatial_orbitals,
        es_problem.num_particles,
        mapper,
    ),
)

vqe_solver = VQE(StatevectorEstimator(), ansatz, SLSQP())
vqe_solver.initial_point = [0.0] * ansatz.num_parameters

In [13]:
from qiskit_nature.second_q.algorithms import GroundStateEigensolver

calc = GroundStateEigensolver(mapper, vqe_solver)

In [14]:
res = calc.solve(es_problem)
print(res)

=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -243.429079135905
  - computed part:      -4.71969517959
  - ActiveSpaceTransformer extracted energy part: -238.709383956314
~ Nuclear repulsion energy (Hartree): 58.291465574664
> Total ground state energy (Hartree): -185.13761356124
 
=== MEASURED OBSERVABLES ===
 
  0:  # Particles: 4.000 S: 0.000 S^2: 0.000 M: 0.000
 
=== DIPOLE MOMENTS ===
 
~ Nuclear dipole moment (a.u.): [0.0  0.0  0.0]
 
  0: 
  * Electronic dipole moment (a.u.): [0.0  0.0  -0.000002326193]
    - computed part:      [0.0  0.0  -0.000002326194]
    - ActiveSpaceTransformer extracted energy part: [0.0  0.0  0.000000000001]
  > Dipole moment (a.u.): [0.0  0.0  0.000002326193]  Total: 0.000002326193
                 (debye): [0.0  0.0  0.000005912592]  Total: 0.000005912592
 
